# <center> SISTEMAS DE INFORMAÇÃO</center>
### <center> MINERAÇÃO DE DADOS</center>
### <center> ATIVIDADE - AULA 04 </center>

# Gerador de Dataset Sintético **sujo** 

Este notebook cria datasets sintéticos com **inconsistências aleatórias** a cada execução:
- valores ausentes
- grafia errada (ruído textual)
- idades inconsistentes
- datas inconsistentes
- situações que exigem **imputação numérica** e **imputação categórica**
- registros duplicados

No final, salva um CSV com **timestamp** (para não sobrescrever) e imprime um **relatório** das sujeiras inseridas.

## Atividade
A atividade dessa aula consiste em gerar um um dataset "sujo", ou seja, com inconsistências e em seguida realizar o seu pré-processamento
Vocês deverão enviar o notebook com os passos executados para realizar o limpeza do dataset
O notebook deverá ter uma célula com um texto explicativo antes da execução de alguma tarefa

## (Opcional) Montar o Google Drive
Execute esta célula se quiser salvar a saída direto no seu Drive. Se não precisar, **pule** e a saída ficará no diretório atual (`/content`).

In [ ]:
USE_DRIVE = False  # Altere para True para salvar no Drive
OUTPUT_DIR = "/content"  # Pasta padrão quando USE_DRIVE=False

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    # Ajuste o caminho abaixo para sua pasta no Drive (ex.: 'MyDrive/datasets')
    OUTPUT_DIR = "/content/drive/MyDrive/datasets"

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Salvar arquivos em: {OUTPUT_DIR}")

## Parâmetros do gerador
- Defina `SEMENTE` como um inteiro para reprodutibilidade.
- Deixe `SEMENTE=None` para aleatoriedade total a cada execução.

In [ ]:
import numpy as np, random

N = 10_000          # número de linhas
SEMENTE = None      # use um inteiro (ex.: 42) para resultados reprodutíveis

if SEMENTE is not None:
    np.random.seed(SEMENTE)
    random.seed(SEMENTE)

print(f"N linhas = {N} | Semente = {SEMENTE}")

## Gerar dataset e injetar inconsistências
A execução cria o DataFrame base (limpo) e então aplica **sujeiras** em porções aleatórias.

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Catálogos simples
NOMES = [
    "Ana","Bruno","Carla","Diego","Eva","Fabio","Gabriela","Heitor","Iris","Joao",
    "Karina","Lucas","Mariana","Nicolas","Olivia","Paulo","Rita","Saulo","Tania","Ulisses",
    "Vera","Will","Xavier","Yasmin","Zeca"
]
SOBRENOMES = ["Silva","Santos","Oliveira","Souza","Lima","Pereira","Almeida","Ferreira","Rodrigues","Carvalho"]
CIDADES = ["sao paulo","rio de janeiro","belo horizonte","curitiba","recife","salvador","manaus","fortaleza","goiania","belem"]
CARGOS = ["analista","cientista de dados","engenheiro de dados","desenvolvedor","estagiario","gestor"]
ESTADOS_CIVIS = ["solteiro","casado","divorciado","viuvo"]

def nome_completo():
    return f"{random.choice(NOMES)} {random.choice(SOBRENOMES)}"

def data_aleatoria(inicio=datetime(2015,1,1), fim=datetime(2025,8,1)):
    delta = fim - inicio
    return inicio + timedelta(days=random.randint(0, delta.days))

# 1) Base limpa
idades = np.clip(np.random.normal(35, 10, N).round().astype(int), 14, 90)
salarios = np.random.lognormal(mean=8.0, sigma=0.45, size=N).round(2)
datas = [data_aleatoria().strftime("%d/%m/%Y") for _ in range(N)]
telefones = [f"(11) 9{random.randint(1000,9999)}-{random.randint(1000,9999)}" for _ in range(N)]

df = pd.DataFrame({
    'nome': [nome_completo() for _ in range(N)],
    'idade': idades,
    'cidade': np.random.choice(CIDADES, N),
    'data_admissao': datas,
    'salario': salarios,
    'estado_civil': np.random.choice(ESTADOS_CIVIS, N),
    'cargo': np.random.choice(CARGOS, N),
    'telefone': telefones
})

# 2) Inconsistências aleatórias
summary = {}

# 2.1 Ausentes
proporcoes_na = {
    'idade': np.random.uniform(0.02, 0.08),
    'salario': np.random.uniform(0.02, 0.08),
    'cidade': np.random.uniform(0.02, 0.08),
    'estado_civil': np.random.uniform(0.01, 0.06),
    'cargo': np.random.uniform(0.01, 0.05),
    'telefone': np.random.uniform(0.60, 0.95)
}
for col, p in proporcoes_na.items():
    m = np.random.rand(N) < p
    df.loc[m, col] = np.nan
summary['na_inseridos_est'] = {k: round(v*N) for k, v in proporcoes_na.items()}

# 2.2 Grafia errada (ruído textual)
def ruidificar_texto(s: str) -> str:
    if not isinstance(s, str):
        return s
    s2 = s
    if random.random() < 0.5:
        s2 = f"  {s2}  "
    if random.random() < 0.5:
        s2 = s2.upper() if random.random() < 0.5 else s2.title()
    mapa = {"o":"0","i":"1","e":"3","a":"4"}
    if random.random() < 0.4:
        for k,v in mapa.items():
            if random.random() < 0.3:
                s2 = s2.replace(k, v)
    if random.random() < 0.5:
        s2 = s2.replace(" ", "  ")
    return s2

for col in ['nome','cidade','estado_civil','cargo']:
    mask_ruido = np.random.rand(N) < np.random.uniform(0.05, 0.15)
    df.loc[mask_ruido, col] = df.loc[mask_ruido, col].astype('object').apply(ruidificar_texto)
summary['ruido_texto_cols'] = ['nome','cidade','estado_civil','cargo']

# 2.3 Idades inconsistentes
idx_bad_idade = np.random.choice(df.index, size=int(np.random.uniform(0.01, 0.03)*N), replace=False)
choices = [-5, -1, 5, 8, 10, 120, 150]
df.loc[idx_bad_idade, 'idade'] = np.random.choice(choices, size=len(idx_bad_idade))
summary['idades_inconsistentes'] = int(len(idx_bad_idade))

# 2.4 Datas inconsistentes (futuras ou lixo)
def data_futura():
    base = datetime(2027,1,1)
    return (base + timedelta(days=random.randint(0, 365*3))).strftime('%d/%m/%Y')
idx_bad_data = np.random.choice(df.index, size=int(np.random.uniform(0.01, 0.03)*N), replace=False)
for i in idx_bad_data:
    df.at[i, 'data_admissao'] = data_futura() if random.random() < 0.7 else 'xx/yy/zzzz'
summary['datas_inconsistentes'] = int(len(idx_bad_data))

# 2.5 Outliers/negativos em salário
sal = pd.to_numeric(df['salario'], errors='coerce')
idx_ruido_sal = np.random.choice(df.index, size=int(np.random.uniform(0.01, 0.03)*N), replace=False)
mult = np.random.choice([10, 20, 0.05], size=len(idx_ruido_sal))
sal.loc[idx_ruido_sal] = sal.loc[idx_ruido_sal] * mult
neg_idx = np.random.choice(df.index, size=int(np.random.uniform(0.003, 0.01)*N), replace=False)
sal.loc[neg_idx] = -abs(sal.loc[neg_idx].fillna(1000))
df['salario'] = sal
summary['salarios_ruido'] = int(len(idx_ruido_sal))
summary['salarios_negativos'] = int(len(neg_idx))

# 2.6 Duplicatas
dup_frac = np.random.uniform(0.01, 0.03)
dups = df.sample(frac=dup_frac, replace=False, random_state=None)
df_sujo = pd.concat([df, dups], ignore_index=True).sample(frac=1.0, random_state=None).reset_index(drop=True)
summary['duplicatas_inseridas'] = int(len(dups))

# 3) Conversões de tipo para facilitar a limpeza
df_sujo['data_admissao'] = pd.to_datetime(df_sujo['data_admissao'], format='%d/%m/%Y', errors='coerce')
df_sujo['idade'] = pd.to_numeric(df_sujo['idade'], errors='coerce')
df_sujo['salario'] = pd.to_numeric(df_sujo['salario'], errors='coerce')

# 4) Relatório rápido
relatorio = {
    'linhas_geradas_total': int(len(df_sujo)),
    'nulos_por_coluna': df_sujo.isna().sum().to_dict(),
    'duplicatas_totais': int(df_sujo.duplicated().sum()),
}
relatorio.update(summary)
relatorio

## Salvar CSV com timestamp
O arquivo é salvo em `OUTPUT_DIR`, definido acima (Drive ou `/content`).

In [ ]:
from datetime import datetime
import os

ts = datetime.now().strftime('%Y%m%d_%H%M%S')
OUT = os.path.join(OUTPUT_DIR, f"dataset_sintetico_sujo_8attrs_{ts}.csv")
df_sujo.to_csv(OUT, index=False)
print(f"✅ CSV salvo em: {OUT}")
df_sujo.head()